In [1]:
import os
import xlrd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import datetime
import statistics as stats
import tsfresh.feature_extraction.feature_calculators as fc
from functools import reduce
import math

### Daten einlesen und Spalten entfernen welche nicht benötigt werden.

In [3]:
reib_daten = pd.read_csv('data_2019-26-11_TRAININGDATA.csv')
# achsleistung_daten = pd.read_csv('../Achsleistungen _ Messprotokolle/Ebene 1/Achsleistung-2019-11-20T10-27-03.csv', \
#                                  delimiter=';')
# messprotokoll_daten = pd.read_csv('../Achsleistungen _ Messprotokolle/Ebene 1/MEAS_PROTOCOL_CSV.CSV', delimiter=';')
messergebnisse_daten = pd.read_excel('../Messergebnisse Reiben/Messergebnisse.xlsx', skiprows=7, delimiter=';')
#achsleistung_columns = ['Timestamp', 'X1', 'Y1', 'Z1', 'SPI1']
#achsleistung_daten = achsleistung_daten[achsleistung_columns]
messergebnisse_daten = messergebnisse_daten.rename(columns={'Messwert Innenmessschraube': 'Messwert Innenmessschraube',\
                                                            'Messwert Messtaster': 'Messwert Messtaster', \
                                                            'Klassifizierung nach Messtaster': 'Klassifizierung nach Messschraube', \
                                                            'Klassifizierung nach Messtaster': 'Klassifizierung nach Messtaster', \
                                                            'Klassifizierung nach Grenzlehrdorn': 'Klassifizierung nach Grenzlehrdorn'})

In [4]:
messergebnisse_daten = messergebnisse_daten.drop(columns=['Unnamed: 6', 'Bezeichnung Bohrung'])
messergebnisse_daten

,Messwert Innenmessschraube,Messwert Messtaster,Klassifizierung nach Messchraube,Klassifizierung nach Messtaster,Klassifizierung nach Grenzlehrdorn
0,12.016,12.019,i.O.,n.i.O.,i.O.
1,12.016,12.018,i.O.,i.O.,i.O.
2,12.016,12.015,i.O.,i.O.,i.O.
3,12.016,12.015,i.O.,i.O.,i.O.
4,12.016,12.015,i.O.,i.O.,i.O.
...,...,...,...,...,...
2344,12.013,NaN,i.O.,n.i.O.,i.O.
2345,12.015,NaN,i.O.,n.i.O.,i.O.
2346,12.014,NaN,i.O.,n.i.O.,i.O.
2347,12.016,NaN,i.O.,n.i.O.,i.O.


### Hilfestellung zur Ermittlung von Löchern
Die Funktion gibt die Intervalle für die gewünschten Bohrungen zurück. Zu beachten:
- keine negativen Zahlen
- Es muss für beide Funktionen dieselbe Ebene gewählt werden, da die Funktion ```bohrungenDavor()```mit 81 Bohrungen weniger rechnet!

In [5]:
def bohrungEbene(ebene):
    aktuelle_bohrung = 81 * ebene
    return aktuelle_bohrung

def bohrungenDavor(ebene):
    if ebene == 0 or ebene == 1:
        return 0
    elif ebene > 1:
        bohrungen_davor = (ebene - 1) * 81
        return bohrungen_davor

    
print("Anz. Bohrungen davor: " + str(bohrungenDavor(2)) + "\nAnz. Bohrungen aktuell: " + str(bohrungEbene(2)))

Anz. Bohrungen davor: 81
Anz. Bohrungen aktuell: 162


### Anpassen und aufbereiten Messprotokolldaten
Da die Maschine die Werte in der Exceltabelle sehr schlecht formatiert - viele Tabulatoren und alle Zahlen sind strings. Zusätzlich werden noch alle null values entfernt.

In [ ]:
# messergebnisse_daten['Messwert Innenmessschraube'] = messergebnisse_daten['Messwert Innenmessschraube'].str.strip()
# messergebnisse_daten['Messwert Innenmessschraube'] = messergebnisse_daten['Messwert Innenmessschraube'].str.replace(',', '.')
# messergebnisse_daten['Messwert Innenmessschraube'] = messergebnisse_daten['Messwert Innenmessschraube'].apply(pd.to_numeric, errors='coerce')

# messergebnisse_daten['Messwert Messtaster'] = messergebnisse_daten['Messwert Messtaster'].str.strip()
# messergebnisse_daten['Messwert Messtaster'] = messergebnisse_daten['Messwert Messtaster'].str.replace(',', '.')
# messergebnisse_daten['Messwert Messtaster'] = messergebnisse_daten['Messwert Messtaster'].apply(pd.to_numeric, errors='coerce')

In [ ]:
# reib_daten = reib_daten.rename(columns=lambda x: x.strip(''))
# messprotokoll_columns = ['Measured', 'Difference']
# messprotokoll_daten = messprotokoll_daten[messprotokoll_columns]

# for column, value in enumerate(messprotokoll_columns):
#     messprotokoll_daten[value] = messprotokoll_daten[value].str.strip()
#     messprotokoll_daten[value] = messprotokoll_daten[value].str.replace(',', '.')
#     messprotokoll_daten[value] = messprotokoll_daten[value].apply(pd.to_numeric, errors='coerce')

# print(messergebnisse_daten.dtypes)

# achsleistung_daten = achsleistung_daten.dropna()
# messprotokoll_daten = messprotokoll_daten.dropna()
# messergebnisse_daten = messergebnisse_daten.dropna()

### Berechnung Mittelwert und Median für Messprotokolldaten

Hier werden die Daten für das Messprotokoll angepasst. Da hier eine Bohrung aus drei Werten besteht (Spalte Result: X, Y, Diameter), werden alle drei Werte genommen und aggregiert $\rightarrow$ Median + Mittelwert.

Danach wird das Messprotokoll mit den Daten der Messergebnisse gemerged.

In [ ]:
# filling_columns = ['Measured Diameter', 'Difference SetpointMeasured']

# messprotokoll_condensed = pd.DataFrame(columns=filling_columns)
# diameter = []
# difference = []
# count = 1
# for key, value in messprotokoll_daten.iteritems():
#     if key == 'Measured':
#         for index, floating in enumerate(value):
#             if (count % 3) == 0:
#                 diameter.append(floating)                
#             if count == 1:
#                 pass
            
#             count += 1

#     elif key == 'Difference':
#         diff = []
#         for index, floating in enumerate(value):
#             if (count % 3) == 0:
#                 #print(value)
#                 difference.append(floating)
                
#             if count == 1:
#                 pass
            
#             count += 1

            

# messprotokoll_condensed['Measured Diameter'] = diameter
# messprotokoll_condensed['Difference SetpointMeasured'] = difference

# #AUFPASSEN, HIER MUSS DIE RICHTIGE EBENE ADRESSIERT WERDEN!

# messergebnisse_ebene = messergebnisse_daten[bohrungenDavor(1):bohrungEbene(1)]

# merge_csv_erg_prot = messergebnisse_ebene.reset_index().merge(messprotokoll_condensed, left_index=True, right_index=True)
# merge_csv_erg_prot

### Achsleitungsverarbeitung und entfernung Ausreiser?
Hier wird die Achleistung bearbeitet. Diskutierbar ist, ob "Ausreiser" die durch Median * 3 ermittelt werden, entfernt werden müssen oder nicht. Es macht keine Unterschiede in der Anzahl der Daten, jedoch in deren Aggregierten Werten (Max, Mean, Median).

In [ ]:
# # delete any row with 0's in SPI
# achsleistung_daten = achsleistung_daten[(achsleistung_daten[['SPI1']] != 0).all(axis=1)]

# # remove median > 3 for all columns
# for key, value in achsleistung_daten.items():
#     if achsleistung_daten[key].dtypes == 'float64':
#         achsleistung_daten.drop(achsleistung_daten[value > (3 * stats.mean(achsleistung_daten[key]))].index, inplace=True)
        

# achsleistung_daten = achsleistung_daten.set_index(['Timestamp'])

In [ ]:
# achsleistung_daten.index = pd.to_datetime(achsleistung_daten.index)
# print(achsleistung_daten.index[-1], achsleistung_daten.index[0])
# time_diff = (achsleistung_daten.index[-1] - achsleistung_daten.index[0])

# resampling_range = time_diff.total_seconds() / 81

# print(math.floor(resampling_range))

# index_series = pd.date_range(start=achsleistung_daten.index[0], end=achsleistung_daten.index[-1], periods=time_diff.total_seconds())

# achsleistung_daten = achsleistung_daten.reindex(achsleistung_daten.index.union(index_series)).interpolate(method='time').reindex(index_series)

# achsleistung_daten
#dateRangeAndAggregate()

### Aufteilung der Daten in gleich große Zeitintervalle und Aggregierung der Daten
Hier wird zunächst die Gesamtzeit zwischen Start und Ende der Achsleistung bestimmt. Diese wird durch 81 geteilt (um einen Zeitraum für eine Bohrung bestimmen zu können), und mit diesem Zeitraum werden die Daten auf verschiedenen Aggregationen - Max, Min, Mean, Median - neu gesampled.
Die berechneten Werte werden dann dem zuvor erstellten DataFrame "merge_csv_erg_prot" hinzugefügt.

Die Datenverarbeitung ist damit abgeschlossen.

In [ ]:
# def reduce_df(dataframe):
#     reduced_frame = reduce(lambda left,right: pd.merge(left,right,on='Timestamp'), dataframe)
#     return reduced_frame

# achsleistung_daten.index = pd.to_datetime(achsleistung_daten.index)

In [ ]:
# time_diff = (achsleistung_daten.index[-1] - achsleistung_daten.index[0])
# resampling_range = (time_diff.total_seconds() / 81)

# print('resampling range: '+ str(resampling_range) + '\ntime difference: ' + str(time_diff) + \
#       '\ntime of one drill: ' + str((time_diff / resampling_range)))

# sampled_mean = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').mean()
# sampled_median = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').median()
# sampled_max = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').max()
# sampled_min = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').min()
# #sampled_app_entropy = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').apply(fc.approximate_entropy(achsleistung_daten), 81, r=1.0)
# #sampled_std_deviation = fc.standard_deviation(achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').std())

# sampled_mean = sampled_mean.rename(
#     columns={'X1': 'X1_mean', 'Y1': 'Y1_mean', 'Z1': 'Z1_mean', 'SPI1': 'SPI1_mean'})
# sampled_median = sampled_median.rename(
#     columns={'X1': 'X1_median', 'Y1': 'Y1_median', 'Z1': 'Z1_median', 'SPI1': 'SPI1_median'})
# sampled_max = sampled_max.rename(                         
#     columns={'X1': 'X1_max', 'Y1': 'Y1_max', 'Z1': 'Z1_max', 'SPI1': 'SPI1_max'})
# sampled_min = sampled_min.rename(                         
#     columns={'X1': 'X1_min', 'Y1': 'Y1_min', 'Z1': 'Z1_min', 'SPI1': 'SPI1_min'})
        
        
# df_list = [sampled_mean, sampled_median, sampled_max, sampled_min]
# df_final = reduce_df(df_list)

# df_final = df_final.reset_index()

# df_final

In [ ]:
# merge_csv_erg_prot = merge_csv_erg_prot.merge(df_final, right_index=True, left_index=True)

In [ ]:
# merge_csv_erg_prot.reset_index(drop=True)


### Bereinigen und hinzufügen Reibdaten

Hier werden die Spalten der Reibdaten mit values, welche mehr als 75% NaN sind, gelöscht. Danach werden die Reibdaten dem Gesamtdataframe hinzugefügt.

In [8]:
reib_daten = reib_daten.rename(columns=lambda x: x.strip('12430012063.'))

def calculatePercentileNaNInReibdaten(column):
    return (count_nans[column] / 81) * 100

count_nans = reib_daten.isna().sum()


for key, value in reib_daten.iteritems():
    if reib_daten[key].dtypes == 'float64':
        if calculatePercentileNaNInReibdaten(key) >= 75:
            reib_daten = reib_daten.drop(columns=key)

In [9]:
machine_data_layer_1 = messergebnisse_daten.merge(reib_daten, left_index=True, right_index=True)

In [10]:
#machine_data_layer_1.columns
machine_data_layer_1 = machine_data_layer_1[['timeStamp', 'Messwert Innenmessschraube',
       'Messwert Messtaster', 'Klassifizierung nach Messchraube',
       'Klassifizierung nach Messtaster', 'Klassifizierung nach Grenzlehrdorn',
       'A1_Axis.Actual_Power', 'Energy_Savings.Active_Power_X',
       'Energy_Savings.Active_Power_Y', 'Energy_Savings.Active_Power_Z',
       'Main_Spindle.MPC_Peak', 'Main_Spindle.MPC_Veff_Total',
       'Main_Spindle.Temperature_Spindle',
       'Tool_Control_Center.Axial_Force_Compression',
       'Tool_Control_Center.Axial_Force_Tension',
       'Tool_Control_Center.Bending_Moment', 'Tool_Control_Center.Symmetry',
       'X1_Axis.Actual_Feed_Rate', 'X1_Axis.Actual_Position_MCS',
       'X1_Axis.Actual_Power', 'Y1_Axis.Actual_Feed_Rate',
       'Y1_Axis.Actual_Position_MCS', 'Y1_Axis.Actual_Power',
       'Y1_Axis.Temperature_Y_Slide', 'Z1_Axis.Actual_Feed_Rate',
       'Z1_Axis.Actual_Position_MCS', 'Z1_Axis.Actual_Power',
       'Label']]

machine_data_layer_1['Main_Spindle.Temperature_Spindle'] = machine_data_layer_1['Main_Spindle.Temperature_Spindle'].fillna(method='ffill')
machine_data_layer_1['Y1_Axis.Temperature_Y_Slide'] = machine_data_layer_1['Y1_Axis.Temperature_Y_Slide'].fillna(method='ffill')
machine_data_layer_1['Tool_Control_Center.Axial_Force_Compression'] = machine_data_layer_1['Tool_Control_Center.Axial_Force_Compression'].fillna(method='pad')
machine_data_layer_1['A1_Axis.Actual_Power'] = machine_data_layer_1['A1_Axis.Actual_Power'].fillna(method='pad')
machine_data_layer_1['Tool_Control_Center.Axial_Force_Tension'] = machine_data_layer_1['Tool_Control_Center.Axial_Force_Tension'].fillna(method='pad')
machine_data_layer_1['Tool_Control_Center.Bending_Moment'] = machine_data_layer_1['Tool_Control_Center.Bending_Moment'].fillna(method='pad')
#machine_data_layer_1.isna().any()


In [12]:
machine_data_layer_1

,timeStamp,Messwert Innenmessschraube,Messwert Messtaster,Klassifizierung nach Messchraube,Klassifizierung nach Messtaster,Klassifizierung nach Grenzlehrdorn,A1_Axis.Actual_Power,Energy_Savings.Active_Power_X,Energy_Savings.Active_Power_Y,Energy_Savings.Active_Power_Z,...,X1_Axis.Actual_Position_MCS,X1_Axis.Actual_Power,Y1_Axis.Actual_Feed_Rate,Y1_Axis.Actual_Position_MCS,Y1_Axis.Actual_Power,Y1_Axis.Temperature_Y_Slide,Z1_Axis.Actual_Feed_Rate,Z1_Axis.Actual_Position_MCS,Z1_Axis.Actual_Power,Label
0,2019-11-26 13:07:26.190,12.016,12.019,i.O.,n.i.O.,i.O.,6.0,9.293467,5.216615,13.801600,...,351.650588,3.214286,-51.928571,-410.568125,2.235294,26.745,529.000000,-295.8600,13.538462,IO
1,2019-11-26 13:07:30.992,12.016,12.018,i.O.,i.O.,i.O.,5.5,9.981437,7.963312,7.460167,...,366.341579,3.470588,-11.812500,-410.535625,2.066667,26.745,0.000000,-295.1650,14.222222,IO
2,2019-11-26 13:07:35.830,12.016,12.015,i.O.,i.O.,i.O.,5.5,10.376278,6.165154,19.023800,...,380.893000,3.882353,-52.714286,-410.543529,2.055556,26.745,4.833333,-295.2850,12.700000,IO
3,2019-11-26 13:07:40.824,12.016,12.015,i.O.,i.O.,i.O.,5.5,9.235176,6.576133,23.935167,...,396.373158,2.764706,-29.714286,-410.529333,3.000000,26.745,434.250000,-295.2300,14.300000,IO
4,2019-11-26 13:07:45.625,12.016,12.015,i.O.,i.O.,i.O.,5.5,10.150611,6.165154,11.190400,...,410.968000,2.411765,0.642857,-410.540000,2.500000,26.745,121.000000,-295.0350,14.000000,IO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,2019-11-26 13:14:03.293,12.015,12.018,i.O.,i.O.,i.O.,5.5,8.200059,12.488538,41.777800,...,410.911500,3.187500,-52.333333,-290.554118,2.000000,26.745,0.000000,-295.5850,13.800000,IO
77,2019-11-26 13:14:08.288,12.015,12.017,i.O.,i.O.,i.O.,5.5,6.528235,6.481308,12.682400,...,426.182632,3.473684,-66.428571,-290.491875,2.277778,26.745,-162.250000,-295.2950,13.928571,IO
78,2019-11-26 13:14:13.123,12.015,12.018,i.O.,i.O.,i.O.,5.5,4.776765,7.113615,8.206200,...,440.857500,2.937500,-24.142857,-290.513750,2.125000,26.745,-175.333333,-295.1525,13.857143,IO
79,2019-11-26 13:14:18.139,12.016,12.018,i.O.,i.O.,i.O.,5.5,4.906125,6.899071,-3.730200,...,456.141579,3.400000,-10.400000,-290.505625,2.250000,26.745,-159.250000,-296.1800,14.000000,IO


In [11]:
# write to csv
machine_data_layer_1.to_csv('./Messergebnisse_merged.csv')
machine_data_layer_1.to_json('./Messergebnisse_merged.json', orient='split')